*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [140]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [141]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-03-11 11:21:48--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip.     [       <=>          ]  24.88M  1.55MB/s    in 16s     

2021-03-11 11:22:05 (1.51 MB/s) - ‘book-crossings.zip.1’ saved [26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [142]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [145]:
#Drop Users that have rewviewed less than 200 books
users_counts = df_ratings["user"].value_counts()
df_ratings_filtered = df_ratings[df_ratings["user"].isin(users_counts[users_counts >= 200].index)]

#Drop books that have left than 100 reviews
ratings_counts = df_ratings["isbn"].value_counts()
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered["isbn"].isin(ratings_counts[ratings_counts >= 100].index)]

df_ratings_filtered.head()

,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0


In [146]:
#Merging both DBs into One and removing Author column as it is not useful
combined_book_ratings = pd.merge(df_ratings_filtered,df_books, on="isbn")
combined_book_ratings = combined_book_ratings.drop(["author"], axis=1)
combined_book_ratings.head()

,user,isbn,rating,title
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...
1,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...
2,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...
3,12538,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...
4,13552,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...


In [147]:
#Counting TotalRatings per books
combined_book_ratings = combined_book_ratings.dropna(axis = 0, subset = ["title"])
book_ratingCount = (combined_book_ratings
                    .groupby( by= ["title"])["rating"]
                    .count()
                    .reset_index()
                    .rename(columns = {"rating": "totalRatingCount"})
                    [["title","totalRatingCount"]]
                    )

book_ratingCount.head()

,title,totalRatingCount
0,1984,52
1,1st to Die: A Novel,162
2,2nd Chance,123
3,4 Blondes,72
4,A Beautiful Mind: The Life of Mathematical Gen...,46


In [155]:
#Merging the count of total Ratings and filtered DB to add the information for later nearest neighbor use
rating_with_totalRatingCount = combined_book_ratings.merge(book_ratingCount, left_on = "title", right_on = "title", how = "left")
rating_with_totalRatingCount.head()

,user,isbn,rating,title,totalRatingCount
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,80
1,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,80
2,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...,80
3,12538,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,80
4,13552,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,80


In [149]:
# Visualizing the spread of the resulting database
print(book_ratingCount["totalRatingCount"].describe())

count    673.000000
mean      73.576523
std       39.844122
min       16.000000
25%       48.000000
50%       63.000000
75%       86.000000
max      365.000000
Name: totalRatingCount, dtype: float64


In [151]:
#Model Definition
rating_with_totalRatingCount = rating_with_totalRatingCount.drop_duplicates(['user', 'title'])
rating_with_totalRatingCount_pivot = rating_with_totalRatingCount.pivot(index = "title", columns = "user", values = "rating").fillna(0)
rating_with_totalRatingCount_matrix = csr_matrix(rating_with_totalRatingCount_pivot.values)

model_knn = NearestNeighbors(metric = "cosine", algorithm = "brute")
model_knn.fit(rating_with_totalRatingCount_matrix)


NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [153]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

  # Find the index of the title passed
  query_index = rating_with_totalRatingCount_pivot.index.get_loc(book) 

  #Find neared neighbor
  distances, indices = model_knn.kneighbors(rating_with_totalRatingCount_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)

  recommended_books = []

  # np.sort(distances)
  for i in range(0, len(distances.flatten())):
      if i == 0:
          recommended_books.append(rating_with_totalRatingCount_pivot.index[query_index])
          recommended_books.append([])
      else:
          recommended_books[1].append([rating_with_totalRatingCount_pivot.index[indices.flatten()[i]],distances.flatten()[i]])

#The model returns 5 items from least to most similar.  Removing the first one since only 4 are required and it is the smallest correlation in there.
  recommended_books[1].remove(recommended_books[1][0])

  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [154]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['I Know This Much Is True', 0.7677075], ['The Surgeon', 0.7699411], ['The Weight of Water', 0.77085835], ["I'll Be Seeing You", 0.8016211]]]
You passed the challenge! 🎉🎉🎉🎉🎉
